In [1]:
import pandas as pd

In [2]:
total_data = pd.read_csv('../combine/FinalJoinedData.csv', index_col=0)

In [3]:
cols_unnamed = []
for col in total_data.columns:
    if 'Unnamed' in col:
        cols_unnamed.append(col)
        
total_data = total_data.drop(columns=cols_unnamed)

In [4]:
total_data.head()

,Seed,Team,Conference,AdjEM,AdjO,AdjD,AdjT,Luck,Opp AdjEM,OppO,...,week11,week12,week13,week14,week15,week16,week17,week18,max,final
0,1,Duke,ACC,34.19,121.0,86.8,74.5,-0.027,9.87,109.1,...,1.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,1.0,1.0
1,1,Cincinnati,CUSA,30.19,118.1,87.9,67.4,0.002,6.58,106.3,...,4.0,4.0,6.0,5.0,4.0,4.0,5.0,5.0,4.0,5.0
2,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,3.0,3.0,3.0,3.0,2.0,2.0,2.0,4.0,2.0,4.0
3,1,Kansas,B12,28.99,118.7,89.7,77.3,0.022,10.66,110.3,...,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0
4,2,Oklahoma,B12,26.04,114.9,88.9,66.5,0.043,8.77,109.0,...,6.0,6.0,4.0,4.0,6.0,5.0,4.0,3.0,3.0,3.0


In [5]:
results = pd.read_csv('../gamesdata/Results.csv', index_col=0)

In [6]:
results.head()

,year,winner,team_one,team_two,score_one,score_two
0,2002,Maryland,Maryland,Siena,85,70
1,2002,Maryland,Maryland,Wisconsin,87,57
2,2002,Maryland,Maryland,Kentucky,78,68
3,2002,Maryland,Maryland,Connecticut,90,82
4,2002,Maryland,Maryland,Kansas,97,88


In [8]:
print (results.shape)

(1112, 6)


In [9]:
def make_predictions(invert=False):
    res = []
    for idx, row in results.iterrows():
        # find the team in our total_data
        curr_year = total_data[total_data['year'] == row['year']]
        curr_team_one = dict(curr_year[curr_year['Team'] == row['team_one']].iloc[0,:])
        curr_team_two = dict(curr_year[curr_year['Team'] == row['team_two']].iloc[0,:])

        result = 1 if row['winner'] == row['team_one'] else 0
        score_one = row['score_one']
        score_two = row['score_two']
        
        curr_row = dict()
        for k, v in curr_team_one.items():
            curr_row[k + '_team_one'] = v
        for k, v in curr_team_two.items():
            curr_row[k + '_team_two'] = v
        curr_row['result'] = result

        curr_row['score_one'] = score_one
        curr_row['score_two'] = score_two
        
        res.append(curr_row)
        
        if invert:
            # experiment with inverting to make more synthetic data
            result = 0 if row['winner'] == row['team_one'] else 1
            curr_row = dict()
            for k,v in curr_team_two.items():
                curr_row[k + '_team_one'] = v
            for k, v in curr_team_one.items():
                curr_row[k + '_team_two'] = v
            curr_row['result'] = result
            curr_row['score_one'] = score_two
            curr_row['score_two'] = score_one

            res.append(curr_row)
    return res

In [10]:
prediction_data = make_predictions()
prediction_invert = make_predictions(True)

In [11]:
df = pd.DataFrame(prediction_data)
df_invert = pd.DataFrame(prediction_invert)

In [12]:
df.head()

,Seed_team_one,Team_team_one,Conference_team_one,AdjEM_team_one,AdjO_team_one,AdjD_team_one,AdjT_team_one,Luck_team_one,Opp AdjEM_team_one,OppO_team_one,...,week14_team_two,week15_team_two,week16_team_two,week17_team_two,week18_team_two,max_team_two,final_team_two,result,score_one,score_two
0,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,85,70
1,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,87,57
2,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,10.0,12.0,11.0,12.0,16.0,4.0,16.0,1,78,68
3,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,0.0,0.0,23.0,19.0,10.0,10.0,10.0,1,90,82
4,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1,97,88


In [13]:
df_invert.head()

,Seed_team_one,Team_team_one,Conference_team_one,AdjEM_team_one,AdjO_team_one,AdjD_team_one,AdjT_team_one,Luck_team_one,Opp AdjEM_team_one,OppO_team_one,...,week14_team_two,week15_team_two,week16_team_two,week17_team_two,week18_team_two,max_team_two,final_team_two,result,score_one,score_two
0,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,85,70
1,16,Siena,MAAC,-1.21,100.8,102.0,66.5,-0.070,-4.14,100.1,...,3.0,2.0,2.0,2.0,4.0,2.0,4.0,0,70,85
2,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,87,57
3,8,Wisconsin,B10,13.22,110.7,97.5,63.3,0.049,10.57,108.9,...,3.0,2.0,2.0,2.0,4.0,2.0,4.0,0,57,87
4,1,Maryland,ACC,29.25,119.2,89.9,73.7,0.025,9.88,109.1,...,10.0,12.0,11.0,12.0,16.0,4.0,16.0,1,78,68


In [14]:
df.to_csv('PredictionData.csv', index=False)

In [15]:
df_invert.to_csv('PredictionDataInvert.csv', index=False)